In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pybedtools
import numpy as np
import pyBigWig
import os
import sys
import gzip
from biodata.bed import BED3Reader
from genomictools import GenomicCollection
from genomictools import GenomicPos
from collections import deque
from scipy import stats
from biodata.delimited import DelimitedReader, DelimitedWriter
from biodata.bed import BEDGraphReader
import builtins
# from biodata.gff import GTFReader

### Generating data & combine

In [2]:
### Helper functions
def generate_grace_period_bins(file, gSize):
  """
  Helper function for grace_period_bins.
  """
#   print(len(file))
  new_rows = []
  for _, row in file.iterrows():
    for i_sta in range(0,gSize+1):
      for i_end in range(0, gSize+1):
        # new_row = {0: row[0], 1: row[1]+i_sta, 2: row[2]+i_end, 3: row[3], 4: row[4], 5: row[5], 6: row[6], 7: row[7]}
        # new_rows.append(new_row)
        
        # new_row = {0: row[0], 1: row[1]-i_sta, 2: row[2]+i_end, 3: row[3], 4: row[4], 5: row[5], 6: row[6], 7: row[7]}
        # new_rows.append(new_row)

        # new_row = {0: row[0], 1: row[1]+i_sta, 2: row[2]-i_end, 3: row[3], 4: row[4], 5: row[5], 6: row[6], 7: row[7]}
        # new_rows.append(new_row)
      
        # new_row = {0: row[0], 1: row[1]-i_sta, 2: row[2]-i_end, 3: row[3], 4: row[4], 5: row[5], 6: row[6], 7: row[7]}
        # new_rows.append(new_row)
        new_row = {0: row[0], 1: row[1]+i_sta, 2: row[2]+i_end, 3: row[3]} # for EnhancerNet
        new_rows.append(new_row)
        
        new_row = {0: row[0], 1: row[1]-i_sta, 2: row[2]+i_end, 3: row[3]}
        new_rows.append(new_row)

        new_row = {0: row[0], 1: row[1]+i_sta, 2: row[2]-i_end, 3: row[3]}
        new_rows.append(new_row)
      
        new_row = {0: row[0], 1: row[1]-i_sta, 2: row[2]-i_end, 3: row[3]}
        new_rows.append(new_row)

  file = pd.concat([file, pd.DataFrame(new_rows)], ignore_index=True).drop_duplicates()
#   print(len(file))
  print("finished generating reference")
  return file


def grace_period_bins(bedfile,gSize):
  """
  Generate a sorted dataframe of binned fragments for respected bedfiles.
  Modified: Use 5bp grace period bins for all references. This means that the regions 
  with at most 5bp differences with the original regions are all counted as 
  possible bins.
  """
  file = pybedtools.BedTool(bedfile).to_dataframe(disable_auto_names=True, header=None)
  return generate_grace_period_bins(file, gSize).sort_values(by=[0, 1, 2])


def get_reference(path, type, idx):
    """ 
    Return the orientation-separated counts file in a dataframe for respective replicate.
    """
    df2 = pybedtools.BedTool(path+type+idx+"/all/count.bed.gz").to_dataframe(disable_auto_names=True, header=None)
    f = df2[df2[3] == "+"]
    r = df2[df2[3] == "-"]
    return f, r


### func for align reference with diff grace period bins
def align(ls):
  """
  Align the sorted STARR-seq segments with different grace period bins.
  Return a list that contains 100% sequence coverage file:
  (chr, start, end, counts)
  """
  input = pybedtools.BedTool.from_dataframe(ls[0]) # forward/reverse counts.bed.gz
  file = pybedtools.BedTool.from_dataframe(ls[1]) # binned reference
  # print(file)
  overlap = file.coverage(input, sorted=True, counts=True, f=1.0, r=True, n=48)
  overlap = overlap.to_dataframe(disable_auto_names=True, header=None)
  return overlap


def count_mapped_bins(UMI, counts, ori_ref, ref, data_type, idx, design, orientation):
# def count_mapped_bins(UMI, counts, ori_ref, ref): # for test
  """ 
  Need to go to the original orientation separated counts.bed.gz file 
  to sum up the overlapped bins read counts if necessary (RNA).
  Sum the reads from fragmented bins.

  Parameter UMI: type boolean; True for RNA, False for DNA
  Parameter counts: dataframe (aligned fragmented bins counts) from pybedtools.coverage
  Parameter ori_ref: dataframe (elements regions after design)
  Parameter ref: forward/reverse counts.bed.gz file
  Parameter data_type: string, DNA or RNA
  Parameter idx: string, 1-6 for DNA, 1-3 for RNA
  Parameter design: string, e.g. TSS_b
  Parameter orientation: string, f or r (forward or reverse)
  """
  if UMI: # RNA samples
    ## subset only overlapped elements from pybedtools.coverage
    # counts = counts[counts[8]==1].reset_index(drop=True)
    counts = counts[counts[4]==1].reset_index(drop=True)
    ## replace the reads since the dataset has already been deduplicated with UMI
    ## also merge the fragmented bins through element id
    # counts[8] = ref.merge(counts, on=[0,1,2], how="inner").loc[:,["4_x"]]
    counts["4_y"] = ref.merge(counts, on=[0,1,2], how="inner").loc[:,["4_x"]]
    # counts = counts.loc[:,[0,1,2,3,8]]
    counts = counts.loc[:,[0,1,2,3,"4_y"]]
    counts = counts.groupby(3, as_index=True).agg({0: 'first', 1: 'first', 2: 'first', "4_y": 'sum'})

  else: # DNA samples
    ## subset only overlapped elements from pybedtools.coverage
    # counts = counts.merge(ref, on=[0,1,2], how="inner").loc[:,[0,1,2,"3_x",8]]
    counts = counts.merge(ref, on=[0,1,2], how="inner").loc[:,[0,1,2,"3_x","4_y"]] #for EnhancerNet
    
    ## merge the fragmented bins through element id
    # counts = counts.groupby("3_x", as_index=True).agg({0: 'first', 1: 'first', 2: 'first', 8: 'sum'})
    counts = counts.groupby("3_x", as_index=True).agg({0: 'first', 1: 'first', 2: 'first', "4_y": 'sum'})

  ## replace the values of fragmented bins by original values retrieved from element id
  if not counts.empty:
    merged_df = counts.merge(ori_ref, left_index=True, right_on=3, how='left')
    
    # output = merged_df.loc[:, ["0_y","1_y", "2_y", 8]]
    output = merged_df.loc[:, ["0_y","1_y", "2_y", "4_y"]]
    print(output)
  else:
    output = pd.DataFrame()
  
  # counts.to_csv("test_output_n.bed", sep="\t", index=False, header=False)
  # output.to_csv("/fs/cbsuhy01/storage/yz2676/data/STARR-seq/partial/data/deep_ATAC_STARR/DNA/"+data_type+idx+"_"+orientation+"_"+design+".bed", sep='\t', index=False, header=False)
  output.to_csv("/fs/cbsuhy01/storage/yz2676/data/STARR-seq/enhancer_end/data/deep_ATAC_STARR/"+data_type+idx+"_"+orientation+"_"+design+".bed", sep='\t', index=False, header=False)

In [7]:
### import DNA/RNA sequence
## path for DNA files
# path = "/fs/cbsuhy01/storage/jz855/STARR_seq_dataset/deep_ATAC_STARR/processing_data_v1/out_DNA_no_UMI/"
# path = "/fs/cbsuhy01/storage/jz855/STARR_seq_dataset/WHG_STARR_seq_TR/processing_data_v1/out_DNA_no_UMI/"
## path for RNA files
path = "/fs/cbsuhy01/storage/jz855/STARR_seq_dataset/deep_ATAC_STARR/processing_data_v1/out_RNA_with_UMI/"
# path = "/fs/cbsuhy01/storage/jz855/STARR_seq_dataset/WHG_STARR_seq_TR/processing_data_v1/out_RNA_no_UMI/"

data_type = "RNA"
idx = "3"
forward, reverse = get_reference(path, data_type, idx)

#### Partial elements

In [ ]:
#for partial elements
from multiprocessing import Pool
UMI = True #False for DNA; True for RNA

#1
ref_path = "../../output/divergent_60bp_without_TSS_b.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "TSS_b", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "TSS_b", "r")

#2
ref_path = "../../output/divergent_60bp_without_TSS_p.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "TSS_p", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "TSS_p", "r")

#3
ref_path = "../../output/divergent_60bp_without_TSS_n.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "TSS_n", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "TSS_n", "r")

#4
ref_path = "../../output/divergent_60bp_without_pause_site_b.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "pause_site_b", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "pause_site_b", "r")

#5
ref_path = "../../output/divergent_60bp_without_pause_site_p.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "pause_site_p", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "pause_site_p", "r")

#6
ref_path = "../../output/divergent_60bp_without_pause_site_n.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "pause_site_n", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "pause_site_n", "r")

In [8]:
from multiprocessing import Pool
UMI = True #False for DNA; True for RNA

#1
ref_path = "../../enhancer_end/Enhancer_3p.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "3p", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "3p", "r")

finished generating reference


     0_y       1_y       2_y  4_y
0  chr17  21043339  21043593    1


In [15]:
## deep_ATAC_STARR
# orientation = "_r_"
# design_type = "pause_site_b"
# DNA_path = ["../data/deep_ATAC_STARR/DNA/DNA1"+orientation+design_type+".bed","../data/deep_ATAC_STARR/DNA/DNA2"+orientation+design_type+".bed",\
#             "../data/deep_ATAC_STARR/DNA/DNA3"+orientation+design_type+".bed","../data/deep_ATAC_STARR/DNA/DNA4"+orientation+design_type+".bed",\
#             "../data/deep_ATAC_STARR/DNA/DNA5"+orientation+design_type+".bed","../data/deep_ATAC_STARR/DNA/DNA6"+orientation+design_type+".bed"]
# RNA_path = ["../data/deep_ATAC_STARR/RNA/RNA1"+orientation+design_type+".bed","../data/deep_ATAC_STARR/RNA/RNA2"+orientation+design_type+".bed",\
#             "../data/deep_ATAC_STARR/RNA/RNA3"+orientation+design_type+".bed","../data/deep_ATAC_STARR/RNA/RNA4"+orientation+design_type+".bed"]

## EnhancerNet
orientation = "_r_"
design_type = "3p"
DNA_path = ["../../enhancer_end/data/deep_ATAC_STARR/DNA1"+orientation+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/DNA2"+orientation+design_type+".bed",\
            "../../enhancer_end/data/deep_ATAC_STARR/DNA3"+orientation+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/DNA4"+orientation+design_type+".bed",\
            "../../enhancer_end/data/deep_ATAC_STARR/DNA5"+orientation+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/DNA6"+orientation+design_type+".bed"]
RNA_path = ["../../enhancer_end/data/deep_ATAC_STARR/RNA1"+orientation+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/RNA2"+orientation+design_type+".bed",\
            "../../enhancer_end/data/deep_ATAC_STARR/RNA3"+orientation+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/RNA4"+orientation+design_type+".bed"]

In [13]:
### func for combining results from biological repeats + DNA/RNA
def combine(input_list,condition):
  """
  Combine the alignment files into a big matrix.
  """
  output_1 = input_list[0].merge(input_list[1], how="outer",left_on=[0,1,2],right_on=[0,1,2], suffixes=('_1', '_2'))
  output_2 = output_1.merge(input_list[2], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_3"})
  output_3 = output_2.merge(input_list[3], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_4"})
  output_4 = output_3.merge(input_list[4], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_5"})
  output_5 = output_4.merge(input_list[5], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_6"})
  # print(output_5)
  output_6 = output_5.merge(input_list[6], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_7"})
  output_7 = output_6.merge(input_list[7], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_8"})
  output_8 = output_7.merge(input_list[8], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_9"})
  output_9 = output_8.merge(input_list[9], how="outer",left_on=[0,1,2],right_on=[0,1,2]).rename(columns={3:"3_10"})
  output_9 = output_9.fillna(0)

  output_9 = output_9[output_9[0] != "chrN"]
  ## change the next two lines when combining full elements
  # output_9.to_csv("../data/deep_ATAC_STARR/"+condition[1]+".bed", sep='\t', index=False, header=False)
  output_9.to_csv("../../enhancer_end/data/"+condition[1]+".bed", sep='\t', index=False, header=False)
  # cmds = ["sort -k1,1 -V ../data/deep_ATAC_STARR/"+condition[1]+".bed > ../data/deep_ATAC_STARR/srt"+condition[1]+condition[0]+".bed",\
  #          "rm ../data/deep_ATAC_STARR/"+condition[1]+".bed"]
  cmds = ["sort -k1,1 -V ../../enhancer_end/data/"+condition[1]+".bed > ../../enhancer_end/data/srt"+condition[1]+condition[0]+".bed",\
           "rm ../../enhancer_end/data/"+condition[1]+".bed"]
  
  # output_9.to_csv("../../../SOLARR/activity/data/"+condition+".bed", sep='\t', index=False, header=False)
  # cmds = ["sort -k1,1 -V ../../../SOLARR/activity/data/"+condition+".bed > ../../../SOLARR/activity/data/srt_"+condition+".bed",\
  #          "rm ../../../SOLARR/activity/data/"+condition+".bed"]
  for cmd in cmds:
    os.system(cmd)

In [16]:
ls = DNA_path + RNA_path
input_ls = [] 
for index in range(len(ls)):
    # print(index)
    file = pybedtools.BedTool(ls[index])
    if file == "": # deal with completely nan file
      print("here")
      # need to append the coordinates and zeros as placeholders, or it cannot merge in the next step??
      ph_data = {0:["chrN"],1:[1.0],2:[2.0],3:[0.0]}
      df = pd.DataFrame(ph_data)
      # print(ph_data)
      input_ls.append(df) 
    else:
      output_file = file.to_dataframe(disable_auto_names=True, header=None)
      input_ls.append(output_file)

combine(input_ls,[design_type,orientation])

here
here


#### Full elements

In [ ]:
refer = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)
refer[3] = ["Divergent"+str(i) for i in range(len(refer))]
print(refer) 
refer.to_csv(ref_path, sep='\t', index=False, header=False)

In [ ]:
# for full elements
from multiprocessing import Pool
UMI = True #False for DNA; True for RNA

# ref_path = "../../GRO_cap_PINTS_qpeak_call_117/K562_GROcap_hg38_1.1.7_qpeak_calls_1_divergent_peaks_element_60bp_e.bed"
ref_path = "../../enhancer_end/Enhancer_K562_5p+60_boundaries_pause_site_b.bed"
reference = grace_period_bins(ref_path, 5)
with Pool(10) as p:
  a_forward, a_reverse = p.map(align, [[forward, reference], [reverse, reference]])

ori_ref = pybedtools.BedTool(ref_path).to_dataframe(disable_auto_names=True, header=None)

count_mapped_bins(UMI, a_forward, ori_ref, forward, data_type, idx, "full", "f")
count_mapped_bins(UMI, a_reverse, ori_ref, reverse, data_type, idx, "full", "r")

finished generating reference
        0_y        1_y        2_y  4_y
1047  chr10   73772486   73772842    4
1052  chr10   74825758   74826052    1
1064  chr10   79826479   79826817    1
1088  chr10   92830995   92831262    2
1126  chr10  101060529  101060938    4
...     ...        ...        ...  ...
872    chr1  246724211  246724606    7
911   chr10   14959124   14959504    2
921   chr10   21525992   21526279    4
966   chr10   46217349   46217586    1
989   chr10   59906776   59907081    2

[207 rows x 4 columns]
        0_y        1_y        2_y  4_y
1057  chr10   75235610   75235963    3
1071  chr10   86959078   86959527    2
1165  chr10  110672363  110672654    2
1180  chr10  120851091  120851505    1
1186  chr10  123135035  123135490    2
...     ...        ...        ...  ...
893   chr10    5479482    5479955    1
920   chr10   21525835   21526185    1
932   chr10   24921352   24921652    2
969   chr10   47461283   47461625    1
982   chr10   50623827   50624206    1

[211 rows

In [ ]:
## deep_ATAC_STARR
design_type = "r_full"
# DNA_path = ["../../full/DNA1_"+design_type+".bed","../../full/DNA2_"+design_type+".bed",\
#             "../../full/DNA3_"+design_type+".bed","../../full/DNA4_"+design_type+".bed",\
#             "../../full/DNA5_"+design_type+".bed","../../full/DNA6_"+design_type+".bed"]
# RNA_path = ["../../full/RNA1_"+design_type+".bed","../../full/RNA2_"+design_type+".bed",\
#             "../../full/RNA3_"+design_type+".bed","../../full/RNA4_"+design_type+".bed"]
DNA_path = ["../../enhancer_end/data/deep_ATAC_STARR/DNA1_"+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/DNA2_"+design_type+".bed",\
            "../../enhancer_end/data/deep_ATAC_STARR/DNA3_"+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/DNA4_"+design_type+".bed",\
            "../../enhancer_end/data/deep_ATAC_STARR/DNA5_"+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/DNA6_"+design_type+".bed"]
RNA_path = ["../../enhancer_end/data/deep_ATAC_STARR/RNA1_"+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/RNA2_"+design_type+".bed",\
            "../../enhancer_end/data/deep_ATAC_STARR/RNA3_"+design_type+".bed","../../enhancer_end/data/deep_ATAC_STARR/RNA4_"+design_type+".bed"]

ls = DNA_path + RNA_path
input_ls = []
for index in range(len(ls)):
    file = pybedtools.BedTool(ls[index])
    output_file = file.to_dataframe(disable_auto_names=True, header=None)
    input_ls.append(output_file)
combine(input_ls, design_type)

### Orientation-independent enh

In [ ]:
### code for checking how many full elements have both forward and reverse strand
### then go to rstudio to check how many of them are orientation-independently functioned
# forward = pybedtools.BedTool("../../full/srt_f_full.bed").to_dataframe(disable_auto_names=True, header=None)
forward = pybedtools.BedTool("../../enhancer_end/data/srt_f_full.bed").to_dataframe(disable_auto_names=True, header=None)

reverse = pybedtools.BedTool("../../enhancer_end/data/srt_r_full.bed").to_dataframe(disable_auto_names=True, header=None)

overlap = pd.merge(forward, reverse, how ='inner', on = [0, 1, 2])
fd = overlap.iloc[:,0:13]
rv = overlap.iloc[:,list(range(0, 3)) + list(range(13, 23))]
# print(rv)
# print("full: ", len(overlap))
fd.to_csv("../../enhancer_end/data/srt_f_full.bed", sep='\t', index=False, header=False)
rv.to_csv("../../enhancer_end/data/srt_r_full.bed", sep='\t', index=False, header=False)

In [ ]:
### overlap full elements with deletion files
## first: merge f/r files for full elements and save it
forward = pybedtools.BedTool("../../enhancer_end/data/srt_f_full_e.bed")
forward = forward.to_dataframe(disable_auto_names=True, header=None)

reverse = pybedtools.BedTool("../../enhancer_end/data//srt_r_full_e.bed")
reverse = reverse.to_dataframe(disable_auto_names=True, header=None)

# defined as enhancers if exhibit enhancer activity on both sides
both = pd.merge(forward, reverse, how ='inner', on = [0, 1, 2]).iloc[:,0:3]

forward = pd.merge(forward, both, how ='inner', on = [0, 1, 2])
reverse = pd.merge(reverse, both, how ='inner', on = [0, 1, 2])

overlap = pd.concat([forward,reverse],ignore_index=True).groupby([0,1,2]).sum().reset_index()
print(overlap)
overlap.to_csv("../../enhancer_end/data/srt_full_ori_indep_e.bed", sep='\t', index=False, header=False)

In [11]:
### save the 3p elements
full = pybedtools.BedTool("../../enhancer_end/data/srt_full_ori_indep_e.bed").to_dataframe(disable_auto_names=True, header=None)
ori = pybedtools.BedTool("../../enhancer_end/3p_boundaries.bed").to_dataframe(disable_auto_names=True, header=None)
ori[3]= ori[3]-60
ori[4] = ori[4]+60
test = pd.DataFrame()
test[1] = ori[3]
test[2] = ori[4]
test[3] = ori[1]
test[4] = ori[2]
ori[1] = test[1]
ori[2] = test[2]
ori[3] = test[3]
ori[4] = test[4]

test = full.merge(ori, how="inner", on=[0,1,2]).iloc[:,[0,-2,-1]]
test["name"] = ["Divergent"+str(i) for i in range(len(test))]
print(test)
test.to_csv("../../enhancer_end/Enhancer_3p.bed", sep="\t", index=False, header=False)

       0        3_y        4_y        name
0  chr17   21043339   21043593  Divergent0
1   chr5   43064745   43065040  Divergent1
2   chr7  100867119  100867498  Divergent2


### Merged full elements files for TSS, pause_site and DPR

In [ ]:
## fifth: get the merged full file for pause_site and dpr
full_tss_p = pybedtools.BedTool("../../full/data/filtered/full_TSS_p.bed").to_dataframe(disable_auto_names=True, header=None)

full_tss_n = pybedtools.BedTool("../../full/data/filtered/full_TSS_n.bed").to_dataframe(disable_auto_names=True, header=None)

full_tss_pn = pd.merge(full_tss_p, full_tss_n, how ='outer', on = [0, 1, 2])
# print(full_tss_pn)
output = full_tss_pn.iloc[:,0:13] # for full elements, it doesn't matter whether it's on f/r strand
output.iloc[-3:,:] = full_tss_pn.iloc[23:, list(range(0, 3)) + list(range(13, 23))].values
print(output)

output.to_csv("../../full/data/filtered/full_tss_pn.bed", sep='\t', index=False, header=False)

In [ ]:
## fifth: get the merged full file for pause_site and dpr
full_ps_p = pybedtools.BedTool("../../full/data/filtered/full_pause_site_p.bed")
full_ps_p = full_ps_p.to_dataframe(disable_auto_names=True, header=None)

full_ps_n = pybedtools.BedTool("../../full/data/filtered/full_pause_site_n.bed")
full_ps_n = full_ps_n.to_dataframe(disable_auto_names=True, header=None)

full_ps_pn = pd.merge(full_ps_p, full_ps_n, how ='outer', on = [0, 1, 2])
# print(full_ps_pn)
output = full_ps_pn.iloc[:,0:13] # for full elements, it doesn't matter whether it's on f/r strand
output.iloc[-2:,:] = full_ps_pn.iloc[32:, list(range(0, 3)) + list(range(13, 23))].values
print(output)

output.to_csv("../../full/data/filtered/full_ps_pn.bed", sep='\t', index=False, header=False)

In [ ]:
## same pipeline for dpr
typ = "low"
full_dpr_p = pybedtools.BedTool("../../full/data/filtered/srt_full_DPR_"+typ+"_conf_p.bed")
full_dpr_p = full_dpr_p.to_dataframe(disable_auto_names=True, header=None)

full_dpr_n = pybedtools.BedTool("../../full/data/filtered/srt_full_DPR_"+typ+"_conf_n.bed")
full_dpr_n = full_dpr_n.to_dataframe(disable_auto_names=True, header=None)

full_dpr_pn = pd.merge(full_dpr_p, full_dpr_n, how ='outer', on = [0, 1, 2])
# print(full_dpr_pn)
output = full_dpr_pn.iloc[:,0:13]
output.iloc[-3:] = full_dpr_pn.iloc[5:, list(range(0, 3)) + list(range(13, 23))].values
print(output)

output.to_csv("../../full/data/filtered/full_dpr_"+typ+"_pn.bed", sep='\t', index=False, header=False)

### Quantify orientation by Maximum/minimum TSS
1. First we will check GRO-cap read counts for full elements for corresponding files (reverse the binned elements to original elements), and then save intermediate to_exchange and no_exchange dictionaries for each file.  
For instance, for forward pause site file, it could have elements that are in the reverse dictionary and have less reads that that in the reverse file, and these are saved in the to_exchange dictionary. Otherwise, when it has elements that are not in the reverse file but have less reads than that in the reverse file, we save them inside the no_exchange dictionary.  
For sanity check, the number of to_exchange dictionary should be of the same length for f_to_exchange and r_to_exchange.  

2. Secondly, load in the files for exchange. First we will use intermediate dics to save the elements and exchange those entries in the to_exchange dic. Then we will add in corresponding no_exchange entries and sort them accordingly.  

3. Finally we merge on exch_f, exch_r files, and fill in with NAN values.  

4. Then we will apply the same approach on elements with no design edits (no deletion).  

5. For both sides deletion files, we merge them with preexisting full element files, if the size was no bigger than previous ones, then the previous file that contains the union of _f, _r full elements can be used directly; if not, then we need to generate new files for both full and partial designs. After creating the lastest full element file, we merge on _b and full files, and fill in with NAN values.

In [ ]:
### should have the original full element boundaries before going to the next step:
### if not, run this cell
from reverse_original import revert
file_input = "../../full/data/PROcap_clear/filtered/srt_full_DPR_p.bed"
revert(file_input)

In [ ]:
### sort if necessary - pipeline need re-arrange!!
des = "TSS_n"
cmds = ["sort -k1,1 -V ../data/deep_ATAC_STARR/filtered/"+des+".bed > ../data/deep_ATAC_STARR/filtered/srt_"+des+".bed",\
           "rm ../data/deep_ATAC_STARR/filtered/"+des+".bed",]
for cmd in cmds:
  os.system(cmd)

In [ ]:
des = "full_TSS_b"
cmds = ["sort -k1,1 -V ../../full/data/filtered/"+des+".bed > ../../full/data/filtered/srt_"+des+".bed",\
           "rm ../../full/data/filtered/"+des+".bed",]
for cmd in cmds:
  os.system(cmd)

In [ ]:
### third: check total GRO-cap read counts from p/n files, see if they need to be exchanged
### this applies not only to TSS analysis, bc we are calculating the maximum/minimum TSS concept
## some divergent elements are not that convincing, might need to delete them
bw1 = pyBigWig.open("../../../GROcap/K562_GROcap_hg38_aligned_pl.bw")
bw2 = pyBigWig.open("../../../GROcap/K562_GROcap_hg38_aligned_mn.bw")
# bw1 = pyBigWig.open("../../20231024_PROcap/alignmentmerge_C1a_and_C1b_pl.bw") # for PROcap
# bw2 = pyBigWig.open("../../20231024_PROcap/alignmentmerge_C1a_and_C1b_mn.bw")

# binned full elements need to find their corresponding original boundaries
# check both positive deletion and negative
des = "TSS"
fp = open("../../full/data/filtered/srt_full_"+des+"_p.bed", "r") # change to TSS/ps accordingly
fr = open("../../full/data/filtered/srt_full_"+des+"_n.bed", "r")
# fp = open("../../full/data/PROcap_clear/filtered/original_full_dpr_p.bed", "r") # change to TSS/ps/dpr accordingly
# fr = open("../../full/data/PROcap_clear/filtered/original_full_dpr_n.bed", "r")
line = fp.readline()
fp_ls = []
while line:
    ls = line.strip("\n").split("\t")
    fp_ls.append(ls)
    line = fp.readline()

fn_ls = []
line1 = fr.readline()
while line1:
    ls = line1.strip("\n").split("\t")
    fn_ls.append(ls)
    line1 = fr.readline()

# need to count in whether they have corresponding negative strand
# save the paired elements into a dictionary, can use index to extract
count = 0
f_dic_to_exchange = {}
f_dic_no_exchange = {}
while count < len(fp_ls):
    p = fp_ls[count]
    chr = p[0]
    start = int(p[1])
    end = int(p[2])
    if np.mean(np.nan_to_num(bw1.values(chr, start, end))) < abs(np.mean(np.nan_to_num(bw2.values(chr, start, end)))):
        if p in fn_ls:
            f_dic_to_exchange[count] = p
        else:
            f_dic_no_exchange[count] = p
    count += 1

count = 0
r_dic_to_exchange = {}
r_dic_no_exchange = {}
while count < len(fn_ls):
    n = fn_ls[count]
    chr = n[0]
    start = int(n[1])
    end = int(n[2])
    if np.mean(np.nan_to_num(bw1.values(chr, start, end))) < abs(np.mean(np.nan_to_num(bw2.values(chr, start, end)))):
        if n in fp_ls:
            r_dic_to_exchange[count] = n
        else:
            r_dic_no_exchange[count] = n
    count += 1

print(f_dic_to_exchange.keys())
print(r_dic_to_exchange.keys())
assert list(f_dic_to_exchange.values()) == list(r_dic_to_exchange.values()), "To_exchange dic of different length"
print(len(f_dic_no_exchange), f_dic_no_exchange.keys())
print(len(r_dic_no_exchange), r_dic_no_exchange.keys())

In [ ]:
### exchange the positive and negative strand data in to_exchange list
### save the no_exchange to their corresponding f/r file
### as printed in the previous cell, there are no elements in the negative strand that needs to save in the other file
des = "TSS"
fo = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_"+des+"_p.bed") # change to TSS/pause_site/dpr
fo = fo.to_dataframe(disable_auto_names=True, header=None)

re = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_"+des+"_n.bed")
re = re.to_dataframe(disable_auto_names=True, header=None)

# Create new DataFrames with rows exchanged
exchanged_df1 = fo.copy()
exchanged_df2 = re.copy()

exchanged_df1.iloc[list(f_dic_to_exchange.keys())] = re.iloc[list(r_dic_to_exchange.keys())].values
exchanged_df2.iloc[list(r_dic_to_exchange.keys())] = fo.iloc[list(f_dic_to_exchange.keys())].values

# print(exchanged_df1)
# Delete the rows from df1 and add them to df2 if in no_exchange dictionary
exchanged_df1 = exchanged_df1[~fo.index.isin(list(f_dic_no_exchange.keys()))].dropna()
exchanged_df2 = exchanged_df2[~re.index.isin(list(r_dic_no_exchange.keys()))].dropna()

exchanged_df2 = pd.concat([exchanged_df2, fo.iloc[list(f_dic_no_exchange.keys())]])
exchanged_df1 = pd.concat([exchanged_df1, re.iloc[list(r_dic_no_exchange.keys())]])
print(len(exchanged_df1))
print(len(exchanged_df2))

# Save the elements
exchanged_df1.to_csv("../data/deep_ATAC_STARR/filtered/exch_"+des+"_p.bed", sep='\t', index=False, header=False)
exchanged_df2.to_csv("../data/deep_ATAC_STARR/filtered/exch_"+des+"_n.bed", sep='\t', index=False, header=False) # not sorted after adding to the end

cmds = ["sort -k1,1 -V ../data/deep_ATAC_STARR/filtered/exch_"+des+"_n.bed > ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_min.bed",\
           "rm ../data/deep_ATAC_STARR/filtered/exch_"+des+"_n.bed",\
        "sort -k1,1 -V ../data/deep_ATAC_STARR/filtered/exch_"+des+"_p.bed > ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_max.bed",\
           "rm ../data/deep_ATAC_STARR/filtered/exch_"+des+"_p.bed",]
for cmd in cmds:
  os.system(cmd)

In [ ]:
### merge p/n, save NAN if they don't have the corresponding element
exchanged_df1 = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_max.bed")
exchanged_df1 = exchanged_df1.to_dataframe(disable_auto_names=True, header=None)

exchanged_df2 = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_min.bed")
exchanged_df2 = exchanged_df2.to_dataframe(disable_auto_names=True, header=None)

output = pd.merge(exchanged_df1, exchanged_df2, how ='outer', on = [0, 1, 2])
df1 = output.iloc[:,0:13]
df2 = output.iloc[:,list(range(0, 3)) + list(range(13, 23))]
print(len(df1))

# Fill NAN values with 0 (for first try)
df1 = df1.fillna(0)
df2 = df2.fillna(0)

df1.to_csv("../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_p.bed", sep='\t', index=False, header=False) # unsorted
df2.to_csv("../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_n.bed", sep='\t', index=False, header=False)

cmds = ["rm ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_min.bed",\
        "rm ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_max.bed",]
for cmd in cmds:
  os.system(cmd)

In [ ]:
# sorting
cmds = ["sort -k1,1 -V ../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_p.bed > ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_max.bed",\
        "rm ../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_p.bed",\
        "sort -k1,1 -V ../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_n.bed > ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_min.bed",\
        "rm ../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_n.bed",]
for cmd in cmds:
  os.system(cmd)

In [ ]:
### For full elements, we will just load in previous merged files and sort them
cmds = ["sort -k1,1 -V ../../full/data/filtered/full_tss_pn.bed > ../../full/data/filtered/srt_full_tss.bed",\
        "rm ../../full/data/filtered/full_tss_pn.bed"]
for cmd in cmds:
  os.system(cmd)

In [ ]:
### Check if _b elements all contained in full
des = "TSS"
b = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_"+des+"_b.bed")
b = b.to_dataframe(disable_auto_names=True, header=None)

full = pybedtools.BedTool("../../full/data/filtered/srt_full_"+des+".bed")
full = full.to_dataframe(disable_auto_names=True, header=None)

output = pd.merge(b, full, how ='outer', on = [0, 1, 2])
df1 = output.iloc[:,0:13]
# print(df1)
df1 = df1.fillna(0)
df1.to_csv("../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_b.bed", sep='\t', index=False, header=False) # unsorted
cmds = ["sort -k1,1 -V ../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_b.bed > ../data/deep_ATAC_STARR/filtered/srt_exch_"+des+"_b.bed",\
        "rm ../data/deep_ATAC_STARR/filtered/srt_t_exch_"+des+"_b.bed",]
for cmd in cmds:
  os.system(cmd)

if len(df1) != len(full): # new element in _b not in _n/_p
  print("not equal")
  # f = pd.merge(full, df1, how ='outer', on = [0, 1, 2]).iloc[:,0:13].fillna(0)

  # p = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_exch_ps_p.bed")
  # p = p.to_dataframe(disable_auto_names=True, header=None)
  # n = pybedtools.BedTool("../data/deep_ATAC_STARR/filtered/srt_exch_ps_n.bed")
  # n = n.to_dataframe(disable_auto_names=True, header=None)

  # df2 = pd.merge(p, df1, how ='outer', on = [0, 1, 2]).iloc[:,0:13].fillna(0)
  # df3 = pd.merge(n, df1, how ='outer', on = [0, 1, 2]).iloc[:,0:13].fillna(0)

  # f.to_csv("../../full/data/filtered/srt_full_t_dpr.bed", sep='\t', index=False, header=False) # unsorted
  # df2.to_csv("../data/deep_ATAC_STARR/filtered/srt_t_exch_dpr_p.bed", sep='\t', index=False, header=False) # unsorted
  # df3.to_csv("../data/deep_ATAC_STARR/filtered/srt_t_exch_dpr_n.bed", sep='\t', index=False, header=False) # unsorted

  # # print(f)
  # # print(df2)
  # cmds = ["sort -k1,1 -V ../../full/data/PROcap_clear/filtered/srt_full_t_dpr.bed > ../../full/data/PROcap_clear/filtered/srt_full_dpr.bed",\
  #       "rm ../../full/data/PROcap_clear/filtered/srt_full_t_dpr.bed",\
  #         "sort -k1,1 -V ../data/PROcap_clear/filtered/srt_t_exch_dpr_p.bed > ../data/PROcap_clear/filtered/srt_exch_dpr_p.bed",\
  #       "rm ../data/PROcap_clear/filtered/srt_t_exch_dpr_p.bed",\
  #         "sort -k1,1 -V ../data/PROcap_clear/filtered/srt_t_exch_dpr_n.bed > ../data/PROcap_clear/filtered/srt_exch_dpr_n.bed",\
  #       "rm ../data/PROcap_clear/filtered/srt_t_exch_dpr_n.bed"]
  # for cmd in cmds:
  #   os.system(cmd)
  # print(f)
  # raise Exception("Not implemented yet")